In [1]:
import pandas as pd
import numpy as np

In [2]:
path_to_casn_smiles = "./casn_to_smiles.csv.gz"
path_to_code_casn = "./code_to_casn.csv.gz"
path_to_hitc_code = "./code_to_hitc.csv.gz"

In [3]:
casn_smiles_df = pd.read_csv(path_to_casn_smiles)
code_casn_df = pd.read_csv(path_to_code_casn)
hitc_code_df = pd.read_csv(path_to_hitc_code)

In [4]:
casn_smiles_df = casn_smiles_df[['Substance_CASRN', 'Structure_SMILES']]
code_casn_df = code_casn_df[['casn', 'code']]
hitc_code_df.rename(columns = {'Unnamed: 0': 'code'}, inplace = True)
casn_smiles_df.rename(columns = {'Substance_CASRN': 'casn', 'Structure_SMILES': 'smiles'}, inplace = True)

In [5]:
code_casn_df.set_index('code', inplace = True)
casn_smiles_df.set_index('casn', inplace= True)

In [6]:
badCounter = 0
for index, data in hitc_code_df.iterrows():
    rowList = data.values.tolist()
    code = rowList[0]
    #get corresponding casn
    try:
        casn = code_casn_df.loc[code, 'casn']
    except KeyError:
        badCounter+=1
        pass
    #get corresponding smiles
    try:
        smiles = casn_smiles_df.loc[casn, 'smiles']
    except KeyError:
        badCounter+=1
        pass
    #write to cell
    hitc_code_df.loc[index, 'code'] = smiles

In [7]:
hitc_code_df.rename(columns = {'code': 'smiles'}, inplace = True)

In [8]:
hitc_code_df.dropna(subset = ['smiles'], inplace = True) 

In [9]:
hitc_code_df.reset_index(inplace = True, drop = True)

In [10]:
hitc_code_df.replace(-1, np.nan, inplace = True)

In [11]:
hitc_code_df.dropna(axis=1,how='all', inplace = True)


In [12]:
hitc_code_df.dropna(axis = 1, thresh=50, inplace=True)

In [13]:
col = hitc_code_df.iloc[:, 0]

In [14]:
hitc_code_df_539 = hitc_code_df.drop(hitc_code_df.columns[hitc_code_df.apply(lambda col: (1 not in col.value_counts() or 0 not in col.value_counts()) or col.value_counts()[1] < 30 or col.value_counts()[0] < 30)], 
                  axis=1)
hitc_code_df_571 = hitc_code_df.drop(hitc_code_df.columns[hitc_code_df.apply(lambda col: (1 not in col.value_counts() or 0 not in col.value_counts()) or col.value_counts()[1] < 25 or col.value_counts()[0] < 25)], 
                  axis=1)

In [15]:
hitc_code_df_539.insert(0, 'smiles', col)
hitc_code_df_571.insert(0, 'smiles', col)

In [16]:
len(hitc_code_df_571.columns)

571

In [17]:
hitc_code_df_539.to_csv("./tox_cast_539.csv", index=False)
hitc_code_df_571.to_csv("./tox_cast_571.csv", index=False)